In [1]:
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import GoogleGenerativeAI, ChatGoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-pro")

In [2]:
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.agents import initialize_agent
from langchain.agents import Tool

from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

In [3]:
from langchain.chains.openai_functions import convert_to_openai_function
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field, validator

In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By

In [7]:
class BlogPosts(BaseModel):
    """Provides list of all the relevant blog posts and links based on the user query input."""
    query: str = Field(description="list of key words to search the blogpost for.")

def get_post_names(query):
    """Provides list of all the relevant blog posts and links based on the user query input.
    Output can have more than one post titles."""

    # striping off unwanted text if any
    query = query.replace('search terms', '')
    query = query.replace('keywords', '')
    query = query.replace(':', '')
    query = query.replace("'", '')

    posts=[]
    driver = webdriver.Chrome() # Update this with your chromedriver path
    blog_url = "https://sandeeprshah.blogspot.com/"
    blog_url = blog_url+"search?q="+query
    # Open the blog website in the browser
    driver.get(blog_url)

    time.sleep(5)  # Wait for 5 seconds to let the page load

    # Find all the search results (assuming search results are represented as links)
    search_results = driver.find_elements(By.CSS_SELECTOR, 'h3')

    # Extract and print the titles of the search results
    print("Titles and links of blog posts:")
    for result in search_results:
        post_title = result.text
        post_link = result.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
#         print(f"Title: {post_title}")
#         print(f"Link: {post_link}")
#         print()
        post_title_link = post_title+":"+post_link
        posts.append(post_title)

    # Close the browser
    driver.quit()

    return {"response": posts}

In [8]:
get_post_names_tool = Tool(
    name="get_posts",
    func=get_post_names,
    description="Provides list of all the relevant blog posts and the links based on the user query input.",
    args_schema = BlogPosts
)

In [9]:
class SinglePost(BaseModel):
    """Given a blog title - find and open the blog in new browser window."""
    link: str = Field(description="link of the blogpost to open in new window")

def open_blog_post_new_window(link):
    """given link for a blog article - open it in new window"""

    # striping off unwanted text if any
    link = link.replace('search terms', '')
    link = link.replace("'", '')

    posts=[]
    driver = webdriver.Chrome() # Update this with your chromedriver path

    driver.get(link)

    time.sleep(5)  # Wait for 5 seconds to let the page load


    return {"response": "Post opened"}

In [10]:
open_blog_post_tool = Tool(
    name="open_blog_post_new_window",
    func=open_blog_post_new_window,
    description="given link for a blog article - open it in new window",
    args_schema = SinglePost
)

In [11]:
tools = [
    get_post_names_tool,
    open_blog_post_tool
]

In [12]:
PREFIX = "<<SYS>> You are smart that selects a function from list of functions based on user queries.\
Run only one function tool at a time or in one query.<</SYS>>\n"

In [13]:
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True,
    max_iterations = 4,
    agent_kwargs={
        'prefix': PREFIX, 
#         'format_instructions': FORMAT_INSTRUCTIONS,
#         'suffix': SUFFIX
    }
)

/Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [14]:
print(agent.agent.llm_chain.prompt.template)

<<SYS>> You are smart that selects a function from list of functions based on user queries.Run only one function tool at a time or in one query.<</SYS>>


get_posts: Provides list of all the relevant blog posts and the links based on the user query input.
open_blog_post_new_window: given link for a blog article - open it in new window

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [get_posts, open_blog_post_new_window]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [15]:
A = agent.run("Fetch me titles of blog posts relevant to keywords: 'marathon training'. \
I just want the list of titles.\
DO NOT OPEN ANY LINKS.")

/Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I should use the get_posts tool to fetch the list of blog posts based on the user query.
Action: get_posts
Action Input: query='marathon training'Titles and links of blog posts:

Observation: {'response': []}
Thought:The list of blog posts for the query is empty, so I cannot extract the titles.
Final Answer: I am sorry, I cannot provide you with a list of titles for blog posts related to 'marathon training' because there are no posts that match your query.

> Finished chain.
